In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/housing.csv')

Drop columns with multicollinearity.

In [3]:
from sklearn.preprocessing import LabelEncoder

df['ocean_proximity'] = LabelEncoder().fit_transform(df['ocean_proximity'])

In [4]:
df.pop('latitude')
df.pop('total_rooms')
df.pop('total_bedrooms')
df.pop('households')
df.pop('housing_median_age')

0        41.0
1        21.0
2        52.0
3        52.0
4        52.0
         ... 
20635    25.0
20636    18.0
20637    17.0
20638    18.0
20639    16.0
Name: housing_median_age, Length: 20640, dtype: float64

In [5]:
import numpy as np
from scipy.stats import zscore
outliers = df[(np.abs(zscore(df)) > 3).all(axis=1)]

df = df[(np.abs(zscore(df)) <= 3).all(axis=1)]

In [6]:
target = df.pop('median_house_value')

In [7]:
features = df

In [8]:
from sklearn.preprocessing import MinMaxScaler

In [9]:
scaler_feature = MinMaxScaler().fit(features)
features = scaler_feature.transform(features)

In [10]:
target = target.to_numpy()

In [11]:
scaler_target = MinMaxScaler().fit(np.reshape(target, (-1, 1)))

In [12]:
target = scaler_target.transform(np.reshape(target, (-1, 1)))

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=1234)

In [14]:
from sklearn.model_selection import cross_val_score
from statistics import mean

In [15]:
from sklearn.linear_model import LinearRegression

In [16]:
print(mean(cross_val_score(LinearRegression(), x_train, y_train, cv=5, verbose=1, scoring='neg_mean_squared_error')))

-0.02910375833215131


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


In [17]:
from sklearn.linear_model import Ridge

In [18]:
print(mean(cross_val_score(Ridge(), x_train, y_train, cv=5, verbose=1, scoring='neg_mean_squared_error')))

-0.029103887351972352


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


In [19]:
from sklearn.linear_model import SGDRegressor

In [20]:
print(mean(cross_val_score(SGDRegressor(), x_train, np.reshape(y_train, (y_train.shape[0],)), cv=5, verbose=1, scoring='neg_mean_squared_error')))

-0.02934674402775249


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished


In [21]:
from sklearn.linear_model import Lasso

In [22]:
print(mean(cross_val_score(Lasso(random_state=1234, tol=0.001, selection='cyclic', alpha=0.3), x_train, y_train, cv=5, verbose=1, scoring='neg_mean_squared_error')))

-0.051968748304053604


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


In [23]:
from sklearn.gaussian_process import GaussianProcessRegressor

In [24]:
print(mean(cross_val_score(GaussianProcessRegressor(), x_train, y_train, cv=5, verbose=1, scoring='neg_mean_squared_error')))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


-0.019526433824213565


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  1.7min finished


In [25]:
from sklearn.kernel_ridge import KernelRidge

In [26]:
print(mean(cross_val_score(KernelRidge(), x_train, y_train, cv=5, verbose=1, scoring='neg_mean_squared_error')))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


-0.029483978859666148


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   39.7s finished


In [27]:
from sklearn.svm import LinearSVR

In [28]:
print(mean(cross_val_score(LinearSVR(max_iter=100000), x_train, np.reshape(y_train, (y_train.shape[0],)), cv=5, verbose=1, scoring='neg_mean_squared_error')))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


-0.030338948356112703


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.3s finished


In [29]:
from sklearn.ensemble import RandomForestRegressor

In [30]:
print(mean(cross_val_score(RandomForestRegressor(), x_train, np.reshape(y_train, (y_train.shape[0],)), cv=5, verbose=1, scoring='neg_mean_squared_error')))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


-0.01705971009878002


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   11.6s finished


The best is RandomForestRegressor. I will find hyperparameters for it.

In [31]:
from sklearn.model_selection import GridSearchCV

In [37]:
params = {
    'n_estimators': [200, 250, 300, 350, 400, 450, 500],
    'max_depth': [None, 10, 50, 100, 200],
    'min_samples_split': [2, 4, 8, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10, 20]
}

In [ ]:
gsc = GridSearchCV(RandomForestRegressor(), param_grid=params, verbose=3, cv=5, scoring='neg_mean_squared_error')
gsc.fit(x_train, np.reshape(y_train, (y_train.shape[0],)))

Fitting 5 folds for each of 875 candidates, totalling 4375 fits
[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=-0.017 total time=   4.7s
[CV 2/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=-0.017 total time=   4.6s
[CV 3/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=-0.017 total time=   4.6s
[CV 4/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=-0.017 total time=   4.6s
[CV 5/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=-0.016 total time=   4.5s
[CV 1/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=250;, score=-0.017 total time=   5.8s
[CV 2/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=250;, score=-0.017 total time=   5.7s
[CV 3/5] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=250;, 

In [36]:
gsc.best_params_

{'n_estimators': 300}